In [1]:
import numpy as np
import viscofit as vf

# Load .IBW Files

In [2]:
###### THIS IS PLACE HOLDER, MUST LOAD AFM DATA
times = [np.linspace(0, 1, 1000) for i in range(3)]
indentations = [(i + 1) * 100e-9 * t / max(t) for i, t in enumerate(times)]
radii = [(i + 1) * 50e-9 for i in range(len(times))]
forces = [vf.forceMaxwell_LeeRadok(np.array([1e5, 1e6, 1e-3]), t, h, r) for t, h, r in zip(times, indentations, radii)]

# Fit Standard Models

### ----------------
### Maxwell Fit
### ----------------

### maxwellModel(forces, times, indentations, radii, E_logbounds=(1, 9), T_logbounds=(-5, 0)

##### initializes an instance of the maxwellModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a maxwell model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM
##### :param E_logbounds: tuple (float, float) high and low log bound for the elastic elements in the model
##### :param T_logbounds: tuple (float, float) high and low log bound for the time constants in the model

In [3]:
# create a maxwellModel instance
# load the AFM forces, times, indentations, and radii
# define the logbounds for E to be between 4 and 7 -> (10^4, 10^7)
# define the logbounds for T to be between -4 and -2 -> (10^-4, 10^-2)
model1 = vf.maxwellModel(forces=forces, times=times, indentations=indentations, 
                         radii=radii, E_logbounds=(4, 7), T_logbounds=(-4, -2))

### maxwellModel.fit(maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to maxwell model of arbitrary size using a nelder-mead simplex which typically gives good fits rather quickly
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [4]:
# fit the maxwellModel instance to the experimental data using Nelder-Mead
# at most, perform 1000 iterations per fit attempt
# the largest possible model will have 3 arms
# the model will be fit sequentially, starting with a 1 arm, then using the best 1 arm parameters as an initial guess
# for the 2 arm model, and the best 2 arm model for the 3 arm model
# perform 10 fit attempts per model
model1.fit(maxiter=1000, max_model_size=3, fit_sequential=True, num_attempts=10)

C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:164: RuntimeWarning: overflow encountered in exp
  relaxance = - sum(model_params[1::2] / model_params[2::2] * exp(- time_matrix / model_params[2::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:164: RuntimeWarning: overflow encountered in multiply
  relaxance = - sum(model_params[1::2] / model_params[2::2] * exp(- time_matrix / model_params[2::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:222: RuntimeWarning: overflow encountered in square
  sse = sum((self.LR_force(model_params=model_params) - self.force) ** 2, axis=0)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:158: RuntimeWarning: overflow encountered in exp
  relaxance = - sum(model_params[0::2] / model_params[1::2] * exp(- time_matrix / model_params[1::2]), axis=1)
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:158: RuntimeWarning: overflow encountered in multiply
  relaxance = - sum(model_params[0::2] / model_params[1::

{'final_params': array([1.e+05, 1.e+06, 1.e-03]),
 'final_cost': 4.4261913735150936e-38,
 'time': 6.554159641265869,
 'trial_variance': 5.1389459725903105e-25}

### maxwellModel.fit_slow(maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to maxwell model of arbitrary size using simulated annealing with a nelder-mead simplex local search, this is very computationally costly and will take a very long time though typically results in much better fits

##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [5]:
# fit the maxwellModel instance to the experimental data using Simulated Annealing with a Nelder-Mead local search
# at most, perform 500 iterations per fit attempt
# the largest possible model will have 3 arms
# the model will not be fit sequentially, fitting all the parameters at once
# perform 7 fit attempts per model
model1.fit_slow(maxiter=500, max_model_size=3, fit_sequential=False, num_attempts=7)

C:\Users\Windows\anaconda3\envs\mat-phys\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


{'final_params': array([2.42900036e+05, 3.55845810e+05, 2.24859981e-03, 5.01254650e+05,
        5.11729377e-04]),
 'final_cost': 1.0746929275473751e-20,
 'time': 32.81090259552002,
 'trial_variance': 2.1831146087533536e-25}

### ----------------
### Kelvin-Voigt Fit
### ----------------

### kelvinVoigtModel(forces, times, indentations, radii, J_logbounds=(-9, -1), T_logbounds=(-5, 0))

##### initializes an instance of the kelvinVoigtModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a kelvin-voigt model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM
##### :param J_logbounds: tuple (float, float) high and low log bound for the compliance elements in the model
##### :param T_logbounds: tuple (float, float) high and low log bound for the time constants in the model

In [6]:
# create a kelvinVoigtModel instance
# load the AFM forces, times, indentations, and radii
# define the logbounds for J to be between -10 and -5 -> (10^-10, 10^-5)
# define the logbounds for T to be between -5 and -3 -> (10^-5, 10^-3)
model2 = vf.kelvinVoigtModel(forces=forces, times=times, indentations=indentations, 
                             radii=radii, J_logbounds=(-10, -5), T_logbounds=(-5, -3))

### kelvinVoigtModel.fit(maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to kelvin-voigt model of arbitrary size using a nelder-mead simplex which typically gives good fits rather quickly
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [7]:
# fit the kelvinVoigtModel instance to the experimental data using Nelder-Mead
# at most, perform 1250 iterations per fit attempt
# the largest possible model will have 2 arms
# the model will be fit sequentially, starting with a 1 arm, then using the best 1 arm parameters as an initial guess
# for the 2 arm model, and the best 2 arm model for the 3 arm model
# perform 6 fit attempts per model
model2.fit(maxiter=1250, max_model_size=2, fit_sequential=True, num_attempts=6)

C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:394: RuntimeWarning: overflow encountered in exp
  retardance = sum(model_params[2::2] / model_params[3::2] * exp(- time_matrix / model_params[3::2]), axis=1) + model_params[1]
C:\Users\Windows\PycharmProjects\mat-phys\viscofit.py:459: RuntimeWarning: overflow encountered in square
  sse = sum((self.LR_scaled_indentation(model_params=model_params) - self.scaled_indentation) ** 2, axis=0)


{'final_params': array([1.00000000e-10, 1.00000000e-10, 4.34523488e-04]),
 'final_cost': 2.594801407563215e-15,
 'time': 3.370981216430664,
 'trial_variance': 2.794528614941764e-23}

### kelvinVoigtModel.fit_slow(maxiter=1000, max_model_size=4, fit_sequential=True, num_attempts=5)

##### fit experimental force distance curve(s) to kelvin-voigt model of arbitrary size using simulated annealing with a nelder-mead simplex local search, this is very computationally costly and will take a very long time though typically results in much better fits
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param max_model_size: int largest number of arms per maxwell model to test (going larger tends to give poor and unphysical fits)
##### :param fit_sequential: bool whether or not to fit sequentially (cascade fit from previous model as the initial guess of the next) (RECOMMENDED)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [8]:
# fit the kelvinVoigtModel instance to the experimental data using Simulated Annealing with a Nelder-Mead local search
# at most, perform 100 iterations per fit attempt
# the largest possible model will have 5 arms
# the model will not be fit sequentially, fitting all the parameters at once
# perform 3 fit attempts per model
model2.fit_slow(maxiter=100, max_model_size=5, fit_sequential=False, num_attempts=3)

{'final_params': array([1.41206920e-10, 1.07785833e-10, 1.60863499e-04, 5.96726435e-08,
        5.28261016e-05]),
 'final_cost': 6.267063647833056e-12,
 'time': 2.213082790374756,
 'trial_variance': 1.827867428306799e-20}

### ----------------
### Power Law Fit
### ----------------

### powerLawModel(forces, times, indentations, radii, E0_logbounds=(1, 9), a_logbounds=(-5, 0))

##### initializes an instance of the customModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a power law rheology model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM
##### :param E0_logbounds: tuple (float, float) high and low log bound for the compliance elements in the model
##### :param a_logbounds: tuple (float, float) high and low log bound for the time constants in the model

In [9]:
# create a powerLawModel instance
# load the AFM forces, times, indentations, and radii
# define the logbounds for E0 to be between 1 and 11 -> (10^1, 10^11)
# define the logbounds for a to be between -5 and 1 -> (10^-5, 10^1)
model3 = vf.powerLawModel(forces=forces, times=times, indentations=indentations, 
                          radii=radii, E0_logbounds=(1, 11), a_logbounds=(-5, 1))

### powerLawModel.fit(maxiter=1000, num_attempts=5)

##### fit experimental force distance curve(s) to power law rheology model using a nelder-mead simplex which typically gives good fits rather quickly
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [10]:
# fit the powerLawModel instance to the experimental data using Nelder-Mead
# at most, perform 2000 iterations per fit attempt
# perform 9 fit attempts per model
model3.fit(maxiter=2000, num_attempts=9)

{'final_params': array([3.5081198e+08, 1.0000000e-05]),
 'final_cost': 1.7989097700117333e-07,
 'time': 7.545182228088379,
 'trial_variance': 4.250522347020523e-24}

### powerLawModel.fit_slow(maxiter=1000, num_attempts=5)

##### fit experimental force distance curve(s) to power law rheology model using simulated annealing with a nelder-mead simplex local search, this is very computationally costly and will take a very long time though typically results in much better fits
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [11]:
# fit the powerLawModel instance to the experimental data using Simulated Annealing with a Nelder-Mead local search
# at most, perform 500 iterations per fit attempt
# perform 4 fit attempts per model
model3.fit_slow(maxiter=500, num_attempts=4)

{'final_params': array([3.50811978e+08, 1.00000000e-05]),
 'final_cost': 1.7989097700117338e-07,
 'time': 6.531060457229614,
 'trial_variance': 5.892314860816896e-29}

# --------------------------------------
# Fit Custom Defined Models
# --------------------------------------

### Define your own function to be fit to the experimental data

### customModel(forces, times, indentations, radi)

##### initializes an instance of the customModel class used for generating fits, of experimentally obtained force-distance data all belonging to the same sample, to a custom defined rheology model which corresponds to the sample's viscoelastic behavior
##### :param forces: either list of numpy arrays or single numpy array corresponding to the force signals from an AFM
##### :param times: either list of numpy arrays or single numpy array corresponding to the time signals from an AFM
##### :param indentations: either list of numpy arrays or single numpy array corresponding to the indentation signals from an AFM
##### :param radii: either list of floats or single float corresponding to the tip radii of an AFM

In [12]:
# create a customModel instance
# load the AFM forces, times, indentations, and radii
model4 = vf.customModel(forces=forces, times=times, indentations=indentations, radii=radii)

### customModel.fit(function, training_data, bounds, maxiter=1000, num_attempts=5)

##### fit experimental observable of your choice to a custom model for the observable using a nelder-mead simplex which typically gives good fits rather quickly
##### :param function: function for the desired observable to be predicted
##### :param training_data: either numpy array or list of numpy arrays the experimental data to be replicated by the function being trained
##### :param bounds: (n, 2) numpy array of upper and lower bounds: [[lower1, upper1], ... [lowerN, upperN]]
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [13]:
# define the custom function that the customModel class will fit
# in this case, we want to fit a function to predict force, given the radius and indentation signal as well as a
# set of parameters

def custom_force_func(params):
    # VERY IMPORTANT NOTES: 
    # the function MUST take in parameters as an argument and MUST output a single vector observable (i.e. force here)
    # the observables (in this case radius and indentation) MUST come from the observables in the model 
    # possible obeservables to use in creating custom functions: (model4.radii, model4.indentation, model4.force, model4.dts)
    # F( t ) = R * p1 * h( t )^p2
    return model4.radii * params[0] * model4.indentation ** params[1]

In [14]:
# define the bounds
# in the case of the function that we made above: params is a (2, ) numpy array: array([p1, p2])
# thus, the bounds must be in the following form:
# array([[p1 lower bounds, p1 upper bounds], 
#        [p2 lower bounds, p2 upper bounds]])

bounds = np.array([[1, 100], [0, 10]])

In [15]:
# fit the customModel instance to the experimental data using Nelder-Mead
# define the function to fit as the custom_force_func that we made above
# define the training data as the force signal in model4 (we will fit the funciton to this data)
# at most, perform 2000 iterations per fit attempt
# perform 9 fit attempts per model
model4.fit(function=custom_force_func, training_data=model4.force, bounds=bounds, maxiter=2000, num_attempts=9)

<ipython-input-13-3ac5fe4fc551>:11: RuntimeWarning: divide by zero encountered in power
  return model4.radii * params[0] * model4.indentation ** params[1]


{'final_params': array([1.00000000e+02, 3.23669728e-17]),
 'final_cost': 3.1209122964812784e-06,
 'time': 1.5965001583099365,
 'trial_variance': 3.155048982396588e-13}

### customModel.fit_slow(function, training_data, bounds, maxiter=1000, num_attempts=5)
        
##### fit experimental observable of your choice to a custom model for the observable using simulated annealing with a nelder-mead simplex local search, this is very computationally costly and will take a very long time though typically results in much better fits
##### :param function: function for the desired observable to be predicted
##### :param training_data: either numpy array or list of numpy arrays the experimental data to be replicated by the function being trained
##### :param bounds: (n, 2) numpy array of upper and lower bounds: [[lower1, upper1], ... [lowerN, upperN]]
##### :param maxiter: int maximum iterations to perform for each fitting attempt (larger number gives longer run time)
##### :param num_attempts: int number of fitting attempts to make per fit, larger number will give more statistically significant results, but will take longer
##### :return: dict {best_fit, (numpy array of final best fit params), final_cost, (float of final cost for the best fit params), time, (float of time taken to generate best fit)}

In [16]:
# fit the powerLawModel instance to the experimental data using Simulated Annealing with a Nelder-Mead local search
# define the function to fit as the custom_force_func that we made above
# define the training data as the force signal in model4 (we will fit the funciton to this data)
# at most, perform 1000 iterations per fit attempt
# perform 4 fit attempts per model
model4.fit_slow(function=custom_force_func, training_data=model4.force, bounds=bounds, maxiter=1000, num_attempts=4)

<ipython-input-13-3ac5fe4fc551>:11: RuntimeWarning: divide by zero encountered in power
  return model4.radii * params[0] * model4.indentation ** params[1]


{'final_params': array([9.99998234e+01, 7.93662976e-09]),
 'final_cost': 3.1209141832952295e-06,
 'time': 2.57759690284729,
 'trial_variance': 8.089459584136234e-15}

# Generate Maps